IMPORTS PARA O PROGRAMA E NORMALIZAÇÃO DO TITLE


In [25]:
import pandas as pd
import re
df = pd.read_csv('data.csv')
dfp = df[df['amount'] > 0]
print(df)
title_norm = df['title'].str.upper()
print(title_norm)
df_cf = pd.read_csv('categorias_financeiras_organizado.csv')
print(df_cf)
df_usr = pd.read_csv('categorias_financeiras_user.csv')
print(df_usr)
vl = pd.read_csv('valuesmax.csv')
print(vl)
#IMPORTACAO DO PANDAS E DAS LISTAS PYTHON

          date                                 title  amount
0   2025-12-20                Ifd*Samara Alves Jardi   33.59
1   2025-12-20                99food *Aloha A Ai Pen   10.98
2   2025-12-18                 F S de Alencar Sanoll   32.70
3   2025-12-18                    99food *604 Burger   31.49
4   2025-12-14                            Dl*99 Ride    9.35
5   2025-12-13           IOF de compra internacional   25.59
6   2025-12-13                  Pg *Loja Gamers Club   39.90
7   2025-12-13                          Csfloat Inc.  731.30
8   2025-12-12                    Pagamento recebido -756.27
9   2025-12-10                 Ifd*Dede Hamburgueria   20.99
10  2025-12-08                    Pagamento recebido -837.21
11  2025-12-08                Uber *One Membership U   19.90
12  2025-12-07                            Ifood Club    7.95
13  2025-12-06  Amazonmktplc*Romanasoa - Parcela 2/4   54.45
14  2025-12-06  Mercadolivre*Mercadol - Parcela 3/10   65.80
15  2025-12-06          

CRIACAO DO PRIMEIRO MODULO (ADICAO DE FILTROS - INICIALMENTE PRE DEFINIDOS - E ESTIPULACAO DE METAS PARA GASTOS)

In [4]:
def adicionar_categoria_gastos():
    nome = input('Digite o nome do centro de custo desejado')
    titulo = input('Digite a palavra chave que identifica o titulo do gasto no extrato *EX: Corrida do Uber = Uber*')
    nome_padrao = nome.upper().lstrip().rstrip()
    titulo_padrao = titulo.upper().lstrip().rstrip()
    save = 'BOTAO SALVAR'
    cancel = 'BOTAO CANCELAR'

    if df_cf['CATEGORIA'].isin([nome_padrao]).any() or df_usr['CATEGORIA'].isin([nome_padrao]).any() :
        print('Essa categoria ja existe! Voce pode adicionar palavras chave para que seu gasto entre nas categorias padroes')
    elif nome == "" or titulo == "":
        print('Nao é possivel adicionar valores vazios')
    elif nome == "" and titulo == "":
        print('Nao é possivel adicionar valores vazios')
    else:
        nova_categoria = df_usr.loc[len(df_usr)] = [nome_padrao,titulo_padrao]
        print(nova_categoria)
        print(df_usr)
        df_usr.to_csv('categorias_financeiras_user.csv', index=False)

adicionar_categoria_gastos()


Nao é possivel adicionar valores vazios


FILTRANDO CATEGORIAS (PRINTA FATURA SOMENTE COM AS CATEGORIAS PADRAO)

In [29]:
def filtro_categorias_default():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False)]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(soma)

filtro_categorias_default()

          date                                 title  amount
0   2025-12-20                Ifd*Samara Alves Jardi   33.59
1   2025-12-20                99food *Aloha A Ai Pen   10.98
3   2025-12-18                    99food *604 Burger   31.49
4   2025-12-14                            Dl*99 Ride    9.35
9   2025-12-10                 Ifd*Dede Hamburgueria   20.99
11  2025-12-08                Uber *One Membership U   19.90
12  2025-12-07                            Ifood Club    7.95
13  2025-12-06  Amazonmktplc*Romanasoa - Parcela 2/4   54.45
14  2025-12-06  Mercadolivre*Mercadol - Parcela 3/10   65.80
15  2025-12-06                  Kabum - Parcela 8/10   65.90
320.4


FILTRANDO GASTOS GERAIS (RETIRA OS GASTOS FILTRADOS E MOSTRA OS "SEM CATEGORIA")

In [28]:
def filtro_geral():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[~(dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False))]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(soma)

filtro_geral()

         date                        title  amount
2  2025-12-18        F S de Alencar Sanoll   32.70
5  2025-12-13  IOF de compra internacional   25.59
6  2025-12-13         Pg *Loja Gamers Club   39.90
7  2025-12-13                 Csfloat Inc.  731.30
829.49


BUSCANDO APENAS A CATEGORIA ESCOLHIDA

In [7]:
def pesquisar_categoria():

pesquisar_categoria()

IndentationError: expected an indented block after function definition on line 1 (3842997863.py, line 3)

INSERINDO PALAVRAS CHAVE EM CATEGORIAS EXISTENTES

In [ ]:
def palavras_chave_categorias_dflt():
    categorias = input('Digite a categoria existente desejada:')
    categorias_padrao = categorias.upper().lstrip().rstrip()
    if df_cf['CATEGORIA'].isin([categorias_padrao]).any():
        titulo = input('Digite as palavras-chave que deseja incluir para reconhecimento do extrato')
        titulo_padrao = titulo.upper().lstrip().rstrip()
        nova_plchave = df_cf.loc[len(df_cf)] = [categorias_padrao,titulo_padrao]
        print(nova_plchave)
    else:
        print('Essa categoria nao existe! Voce pode criá-la ou incluir em alguma das categorias existentes')

palavras_chave_categorias_dflt()

ESTIPULANDO VALOR MAXIMO PARA CADA CATEGORIA

In [ ]:
def valor_maximo_categoria():
    while True:
        categorias = input('Digite a categoria que deseja configurar:')
        categorias_padrao = categorias.upper().lstrip().rstrip()
        if df_cf['CATEGORIA'].isin([categorias_padrao]).any() or df_usr['CATEGORIA'].isin([categorias_padrao]).any() :
            valor = float(input('Digite o valor maximo que deseja gastar:'))
        else:
            print('Essa categoria nao existe!')
            return False
        
        if vl['CATEGORIA'].isin([categorias_padrao]).any():
            print('Voce ja adicionou um valor para essa categoria')
            return False 
        add_valor = vl.loc[len(vl)] = [categorias_padrao,valor]
        print(add_valor)
        vl.to_csv('valuesmax.csv', index=False)
        return False

valor_maximo_categoria()

['ASSINATURA', 50.0]


False

VERIFICADOR DE VALOR POR CATEGORIA

In [ ]:
def verif_val_max():
    print(df_cf['CATEGORIA'].unique(), df_usr['CATEGORIA'].unique())
    print(vl)
    
verif_val_max()

['DELIVERY' 'VAREJO' 'TRANSPORTE' 'MERCADO' 'ENTRETENIMENTO' 'SAUDE'
 'CONTAS'] ['ASSINATURA']
        CATEGORIA   VALOR
0        DELIVERY   500.0
1         CELULAR   150.0
2           SAUDE   500.0
3         MERCADO  1000.0
4  ENTRETENIMENTO   250.0
5      ASSINATURA    50.0


RETIRANDO CATEGORIA DO USUARIO

In [ ]:
def retirar_categoria():
    global df_usr
    while True:
        print(df_usr)
        categoria_usr = input('Qual categoria deseja retirar?').upper().lstrip().rstrip()
        if df_usr['CATEGORIA'].isin([categoria_usr]).any():
            print(df_usr[df_usr['CATEGORIA'] == categoria_usr])
        else:
            print('Essa categoria nao existe!')
            return False
        titulo = input('Digite os titulos que voce deseja remover (caso queira remover toda a categoria, digite TODOS)').upper().lstrip().rstrip()
        if df_usr['TITULO'].isin([titulo]).any():
            index = (df_usr[df_usr['TITULO'] == titulo]).index
            df_usr = df_usr.drop(index)
            df_usr.to_csv('categorias_financeiras_user.csv', index=False)
            print(df_usr)
            return False
        else:
            print('Esse titulo nao existe')
            return False
retirar_categoria()

In [26]:
print(dfp)

          date                                 title  amount
0   2025-12-20                Ifd*Samara Alves Jardi   33.59
1   2025-12-20                99food *Aloha A Ai Pen   10.98
2   2025-12-18                 F S de Alencar Sanoll   32.70
3   2025-12-18                    99food *604 Burger   31.49
4   2025-12-14                            Dl*99 Ride    9.35
5   2025-12-13           IOF de compra internacional   25.59
6   2025-12-13                  Pg *Loja Gamers Club   39.90
7   2025-12-13                          Csfloat Inc.  731.30
9   2025-12-10                 Ifd*Dede Hamburgueria   20.99
11  2025-12-08                Uber *One Membership U   19.90
12  2025-12-07                            Ifood Club    7.95
13  2025-12-06  Amazonmktplc*Romanasoa - Parcela 2/4   54.45
14  2025-12-06  Mercadolivre*Mercadol - Parcela 3/10   65.80
15  2025-12-06                  Kabum - Parcela 8/10   65.90
